## Generating Synthetic Test Datasets

**Why use Synthetic Test Datasets?**

Evaluating the performance of RAG (Retrieval-Augmented Generation) augmented pipelines is crucial.

However, manually creating hundreds of QA (Question-Answer-Context) samples from documents can be time-consuming and labor-intensive. Additionally, human-generated questions may struggle to reach the level of complexity needed for thorough evaluation, ultimately affecting the quality of the assessment.

Using synthetic data generation can reduce developer time in the data aggregation process **by up to 90%**.


In [ ]:
import pdfplumber
from typing import List, Dict, Any

class PDFLoader:
    def __init__(self, file_path: str, start_page: int = None, end_page: int = None):
        self.file_path = file_path
        self.start_page = start_page
        self.end_page = end_page

    def load(self) -> Dict[str, Any]:
        combined_text = ""
        metadata = {}

        with pdfplumber.open(self.file_path) as pdf:
            total_pages = len(pdf.pages)

            start = (self.start_page or 1) - 1
            end = min(self.end_page or total_pages, total_pages)

            for page_num in range(start, end):
                page = pdf.pages[page_num]
                text = page.extract_text()
                combined_text += text + "\n"

            metadata = {
                "source": self.file_path,
                "filename": self.file_path,
                "total_pages": total_pages,
                "extracted_pages": f"{start + 1}-{end}"
            }

            for key, value in pdf.metadata.items():
                if isinstance(value, (str, int)):
                    metadata[key] = value

        return {
            "page_content": combined_text.strip(),
            "metadata": metadata
        }

## Document Used for Practice

Amazon Bedrock Manual Documentation (https://docs.aws.amazon.com/bedrock/latest/userguide/)

- Link: https://d1jp7kj5nqor8j.cloudfront.net/bedrock-manual.pdf
- File name: `bedrock-manual.pdf`

_Please copy the downloaded file to the data folder for the practice session_

In [ ]:
loader = PDFLoader("data/bedrock-manual.pdf", start_page=16, end_page=1574)
docs = loader.load()

## Document Preprocessing

In [ ]:
import re
from typing import List, Optional

def split_text(text: str, chunk_size: int = 1000, chunk_overlap: int = 100, separators: Optional[List[str]] = None) -> List[str]:
    separators = separators or ["\n\n", "\n", " ", ""]

    def _split_text_recursive(text: str, separators: List[str]) -> List[str]:
        if not separators:
            return [text]

        separator = separators[0]
        splits = re.split(f"({re.escape(separator)})", text)
        splits = ["".join(splits[i:i+2]) for i in range(0, len(splits), 2)]

        final_chunks = []
        current_chunk = ""

        for split in splits:
            if len(current_chunk) + len(split) <= chunk_size:
                current_chunk += split
            else:
                if current_chunk:
                    final_chunks.append(current_chunk)
                if len(split) > chunk_size:
                    subsplits = _split_text_recursive(split, separators[1:])
                    final_chunks.extend(subsplits)
                else:
                    current_chunk = split

        if current_chunk:
            final_chunks.append(current_chunk)

        return final_chunks

    chunks = _split_text_recursive(text, separators)

    if chunk_overlap > 0:
        overlapped_chunks = []
        for i, chunk in enumerate(chunks):
            if i == 0:
                overlapped_chunks.append(chunk)
            else:
                overlap_text = chunks[i-1][-chunk_overlap:]
                overlapped_chunks.append(overlap_text + chunk)
        chunks = overlapped_chunks

    return chunks

In [ ]:
chunks = split_text(docs['page_content'], 1000, 0)
len(chunks)

In [ ]:
chunks_with_metadata = []
for i, chunk in enumerate(chunks):
    chunks_with_metadata.append({
        'content': chunk,
        'metadata': {
            'chunk_id': i,
            'filename': docs['metadata'].get('filename', 'unknown')
        }
    })

In [ ]:
chunks_with_metadata[0]

## Test Q&A Dataset Generation

In [ ]:
import boto3
from botocore.config import Config

region = 'us-west-2'
retry_config = Config(
    region_name=region,
    retries={"max_attempts": 10, "mode": "standard"}
)
boto3_client = boto3.client("bedrock-runtime", region_name=region, config=retry_config)

In [ ]:
import random
import json
from time import sleep

def converse_with_bedrock(model_id, sys_prompt, usr_prompt):
    temperature = 0.5
    top_p = 0.9
    inference_config = {"temperature": temperature, "topP": top_p}
    response = boto3_client.converse(
        modelId=model_id,
        messages=usr_prompt, 
        system=sys_prompt,
        inferenceConfig=inference_config,
    )
    return response

def create_prompt(sys_template, user_template):
    sys_prompt = [{"text": sys_template}]
    usr_prompt = [{"role": "user", "content": [{"text": user_template}]}]
    return sys_prompt, usr_prompt

def get_context_chunks(chunks_with_metadata, start_id):
    context_chunks = [
        chunks_with_metadata[start_id]['content'],
        chunks_with_metadata[start_id + 1]['content'],
        chunks_with_metadata[start_id + 2]['content']
    ]
    return " ".join(context_chunks)

### Tool Use 

LLM will generate Q&A dataset that conforms to the schema description in the tooluse config.

In [ ]:
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "QuestionAnswerGenerator",
                "description": "Generates questions and answers based on the given context.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "question": {
                                "type": "string",
                                "description": "The generated question"
                            },
                            "answer": {
                                "type": "string",
                                "description": "The answer to the generated question"
                            }
                        },
                        "required": ["question", "answer"]
                    }
                }
            }
        }
    ]
}

In [ ]:
def converse_with_bedrock_tools(sys_prompt, usr_prompt, tool_config):
    temperature = 0.0
    top_p = 0.1
    top_k = 1
    inference_config = {"temperature": temperature, "topP": top_p}
    additional_model_fields = {"top_k": top_k}
    response = boto3_client.converse(
        modelId="anthropic.claude-3-5-sonnet-20240620-v1:0",
        messages=usr_prompt,
        system=sys_prompt,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields,
        toolConfig=tool_config
    )
    return response

def parse_tool_use(message):
    stop_reason = message['stopReason']

    if stop_reason == 'tool_use':
        tool_requests = message['output']['message']['content']
        for tool_request in tool_requests:
            if 'toolUse' in tool_request:
                tool = tool_request['toolUse']

                if tool['name'] == 'QuestionAnswerGenerator':
                    return tool['input']
    return None

## Q&A Dataset Generation Instruction

- `simple`: directly answerable questions from the given context
- `complex`: reasoning questions and answers.

_Modify the system/user prompts tailored to your dataset_

Generated Q&A pair will be stored in `data/qa_dataset.jsonl`

In [ ]:
def generate_qa_dataset(chunks, num_pairs=5, output_file="data/sample_qa_dataset.jsonl"):
    total_chunks = len(chunks)
    dataset = []

    for i in range(num_pairs):
        start_id = random.randint(0, total_chunks - 3)
        context = get_context_chunks(chunks_with_metadata, start_id)

        if i % 2 == 0:
            sys_template = """
            You are an expert at generating practical questions based on given documentation.
            Your task is to generate complex, reasoning questions and answers.

            Follow these rules:
            1. Generate questions that reflect real user information needs related to the document's subject matter (e.g., technical docs : feature availability, implementation details)
            2. Ensure questions are relevant, concise, preferably under 25 words, and fully answerable with the provided information
            3. Focus on extracting key information that users are likely to seek, while avoiding narrow or less important questions.
            4. When provided with code blocks, focus on understanding the overall functionality rather than the specific syntax or variables. Feel free to request examples of how to use key APIs or features.
            5. Do not use phrases like 'based on the provided context' or 'according to the context'.
            """
            question_type = "complex"
        else:
            sys_template = """
            You are an expert at generating practical questions based on given documentation.
            Your task is to create simple, directly answerable questions from the given context.

            Follow these rules:
            1. Generate questions that reflect real user information needs related to the document's subject matter (e.g., technical docs : feature availability, implementation details)
            2. Ensure questions are relevant, concise, preferably under 10 words, and fully answerable with the provided information
            3. Focus on extracting key information that users are likely to seek, while avoiding narrow or less important questions.
            4. When provided with code blocks, focus on understanding the overall functionality rather than the specific syntax or variables. Feel free to request examples of how to use key APIs or features.
            5. Do not use phrases like 'based on the provided context' or 'according to the context'.
            """
            question_type = "simple"

        user_template = f"""
        Generate a {question_type} question and its answer based on the following context:

        Context: {context}

        Use the QuestionAnswerGenerator tool to provide the output.
        """

        sys_prompt, user_prompt = create_prompt(sys_template, user_template)
        response = converse_with_bedrock_tools(sys_prompt, user_prompt, tool_config)
        qa_data = parse_tool_use(response)

        if qa_data:
            qa_item = {
                "question": qa_data["question"],
                "ground_truth": qa_data["answer"],
                "question_type": question_type,
                "contexts": context
            }

            print(qa_item)

            with open(output_file, 'a') as f:
                json.dump(qa_item, f)
                f.write('\n')

            dataset.append(qa_item)

        sleep(5)

    return dataset

In [ ]:
generate_qa_dataset(chunks_with_metadata, 50)